# Test ReCAPTCHA de Google


Dans cette partie nous utiliser les ressource présent dans le répertoir "test"

In [3]:
# import de libraire

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import urllib2
import os
import cookielib
import json
import time
from collections import Counter

## Partie pour le téléchargement d'images

le but et de télécharger un grand nombre d'images sur google images avec un mot clé définie 
(içi : panneaux de signalisation)

In [18]:
search = "panneaux de signalisation"

def get_soup(url,header):
    return BeautifulSoup(urllib2.urlopen(urllib2.Request(url,headers=header)),'html.parser')



def download(): 
        query= search.split()
        query='+'.join(query)
        url="https://www.google.fr/search?q="+query+"&source=lnms&tbm=isch&tbs=ic:specific"
        #url="https://www.google.fr/search?safe=off&sa=G&hl=fr&tbs=simg:CAESuQIJyN74dJUufBUarQILEKjU2AQaAggLDAsQsIynCBpiCmAIAxIosgHXBtES1ga-ArMB8gb7BdkG9xL6M-s0kSKQKaU0pzTLKJc1lSLtNBowh1PNHrwTUb64ZJ2FwcaGVbmoUzlNEbh-G7abuI4ZY6jFv4etFTgnJqVGiQrQ-0c4IAQMCxCOrv4IGgoKCAgBEgQ0QURGDAsQne3BCRqaAQogCg1iYW5hbmEgZmFtaWx52qWI9gMLCgkvbS8wMTN4azYKGAoGYmFuYW5h2qWI9gMKCggvbS8wOXFjawoiChBjb29raW5nIHBsYW50YWlu2qWI9gMKCggvbS8wOXFkZAofCgtzYWJhIGJhbmFuYdqliPYDDAoKL20vMGc1cHpoXwoXCgVmcnVpdNqliPYDCgoIL20vMDJ4d2IM&q="+query+"&tbm=isch&ved=0ahUKEwi71dvR57DSAhXSSxoKHbBFARgQsw4ILA&biw=1745&bih=841"
        print url
        #add the directory for your image here
        DIR="img"
        header={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"
        }
        soup = get_soup(url,header) #by stack


        ActualImages=[]
        for a in soup.find_all("div",{"class":"rg_meta"}):
            link , Type =json.loads(a.text)["ou"]  ,json.loads(a.text)["ity"]
            ActualImages.append((link,Type))

        print  "there are total" , len(ActualImages),"images"

        if not os.path.exists(DIR):
                    os.mkdir(DIR)
        if not os.path.exists(DIR):
                    os.mkdir(DIR)
        cntr = 0
        for i , (img , Type) in enumerate( ActualImages):
            try:
                req = urllib2.Request(img, headers={'User-Agent' : header})
                raw_img = urllib2.urlopen(req).read()
                if len(Type)==3:
                    cntr = cntr + 1
                    f = open(os.path.join(DIR ,"big"+"Img"+"_"+ str(cntr)+".jpg"), 'wb')
                    f.write(raw_img)
                    f.close()
                    size = 128,128
                    im = Image.open("img/big"+"Img"+"_"+ str(cntr)+".jpg")
                    im.thumbnail(size, Image.ANTIALIAS)
                    im.save("img/"+"Img"+"_"+ str(cntr)+".jpg", "JPEG")



            except Exception as e:
                print "could not load : "+img
                print e
        print "Download finished"

        
download()

https://www.google.fr/search?q=panneaux+de+signalisation&source=lnms&tbm=isch&tbs=ic:specific
there are total 100 images
could not load : http://applications.eduscol.education.fr/D0160/03_034assr12signalisation_pan3.gif
cannot write mode P as JPEG
could not load : http://applications.eduscol.education.fr/D0160/03_034assr12signalisation_pan5.gif
cannot write mode P as JPEG
could not load : http://applications.eduscol.education.fr/D0160/03_034assr12signalisation_pan1.gif
cannot write mode P as JPEG
could not load : http://www.expression-anglaise.com/wp-content/uploads/2015/04/interdiction-demi-tour.gif
cannot write mode P as JPEG
could not load : http://www.hellopro.fr/images/produit-2/2/5/0/panneau-de-signalisation-b2b-1299052.gif
cannot write mode P as JPEG
could not load : http://www.hellopro.fr/images/produit-2/4/9/0/panneau-de-signalisation-b21c1-1299094.gif
cannot write mode P as JPEG
could not load : http://www.hellopro.fr/images/produit-2/7/0/2/panneau-de-signalisation-c107-12992

## Découpage de l'image

l'image du test de google et en fait une seul image qu'il faut découpé découpé (içi 4 x 4)

## Classification d'immages

le but de cette dernier partie et de dire si oui ou non les images découpé ce raproche d'un panneaux de signalisation avec un aproximation de environ 70% étant donner la simpliciter de l'images donné)